In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime

In [2]:
driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
url = "https://jw-marriott.marriott.com/destinations/"
driver.get(url)
time.sleep(30)
# viewhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[3]/button')[0]
# viewhotels_button.click()
# northamhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[2]/pacsys-directory-listings/div/div[5]/a')[0]
# northamhotels_button.click()

In [3]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [4]:
stuff = soup.find_all('a', {'class', 'swell_vertical-xs'})
# stuff
hotel_list = []
for x in stuff:
    hotel = {}
    hotel['name'] = x.text.strip()
    hotel['url'] = x.attrs['href']
    hotel_list.append(hotel)
achotels_df = pd.DataFrame(hotel_list)
achotels_df

,name,url
0,JW Marriott Hotel Cairo,https://www.marriott.com/CAIJW
1,JW Marriott Absheron Baku,https://www.marriott.com/GYDJW
2,JW Marriott Hotel Zhejiang Anji,https://www.marriott.com/HGHAJ
3,JW Marriott Hotel Beijing,https://www.marriott.com/BJSJW
4,JW Marriott Hotel Beijing Central,https://www.marriott.com/BJSJC
...,...,...
98,JW Marriott Hotel Rio de Janeiro,https://www.marriott.com/RIOMC
99,JW Marriott Hotel Bogota,https://www.marriott.com/BOGJW
100,JW Marriott Hotel Quito,https://www.marriott.com/UIODT
101,JW Marriott El Convento Cusco,https://www.marriott.com/CUZMC


In [5]:
%%time

hotel_list = []
counter = 0
for index, row in achotels_df.iterrows():
    name = row['name']
    url = row['url']
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        stuff = soup.find('div', {'class' : 'l-container l-header-container'})
#             hotel_list = []
        hotel = {}
        try:
            hotel['brand'] = 'JW Marriott'
        except:
            hotel['brand'] = 'JW Marriott'
        try:
            hotel['name'] = name
        except:
            hotel['name'] = name
        try:
            hotel['url'] = url
        except:
            hotel['url'] = url
        try:
            hotel['street'] = stuff.find_all('span')[2].text
        except:
            hotel['street'] = 'N/A'
        try:
            hotel['locality'] = stuff.find_all('span')[4].text
        except:
            hotel['locality'] = 'N/A'        
        try:
            hotel['state'] = stuff.find_all('span')[6].text
        except:
            hotel['state'] = 'N/A'            
        try:
            hotel['postalcode'] = stuff.find_all('span')[7].text
        except:
            hotel['postalcode'] = 'N/A'            
        try:
            hotel['coordinate1'] = stuff.find_all('span')[25].text
        except:
            hotel['coordinate1'] = 'N/A'            
        try:
            hotel['coordinate2'] = stuff.find_all('span')[26].text
        except:
            hotel['coordinate2'] = 'N/A'
        try:
            hotel['coordinate3'] = stuff.find_all('span')[27].text
        except:
            hotel['coordinate3'] = 'N/A'            
        try:
            hotel['coordinate4'] = stuff.find_all('span')[28].text
        except:
            hotel['coordinate4'] = 'N/A'            
        
        desc = soup.find_all('p', {'class' : 'l-margin-top-none l-overflow-hidden t-collapsed-height'})
        try:
            hotel['description'] = str(desc).replace('<p class="l-margin-top-none l-overflow-hidden t-collapsed-height">','').replace('</p>', '')
        except:
            hotel['description'] = 'N/A'
        
        overallratings = stuff = soup.find('div', {'class' : 'l-overall-rating-container'})
        try:
            hotel['overall_rating'] = ratings.find_all('div')[0].text.strip()[:4]
        except:
            hotel['overall_rating'] = 'N/A'
        
        specratings = stuff = soup.find('div', {'class' : 'l-secondary-ratings-container l-clear l-margin-bottom-quarter l-m-margin-bottom-none'})
        try:
            hotel['cleanliness_rating'] = specratings.find_all('div')[2].text.strip()[:4]
        except:
            hotel['cleanliness_rating'] = 'N/A'
        try:
            hotel['service_rating'] = specratings.find_all('div')[7].text.strip()[:4]
        except:
            hotel['service_rating'] = 'N/A'
            
        hotel_list.append(hotel)
        print(hotel)
        print()
        print(datetime.now())
        print('='*80)
        print('='*80)
        print()
#         driver.close()
#         driver.switch_to.window(original_window)
#         time.sleep(1)

        counter += 1
        print(f'Collected {counter} hotels.')
    except:
        hotel = {}
        hotel['name'] = name
        hotel['url'] = url
        hotel_list.append(hotel)
        print(f"Number {x} didn't work at {datetime.now()}")
        continue
hotel_df = pd.DataFrame(hotel_list)
hotel_df  

{'brand': 'JW Marriott', 'name': 'JW Marriott Hotel Cairo', 'url': 'https://www.marriott.com/CAIJW', 'street': 'Ring Road- Mirage City- P.O.Box 427, Heliopolis', 'locality': 'Cairo', 'state': '', 'postalcode': '11757', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/caijw-jw-marriott-hotel-cairo/?directPageRequest=true', 'coordinate3': '30.071958', 'coordinate4': '31.43415', 'description': "[ The luxury JW Marriott Hotel Cairo offers a truly immersive experience, with five-star accommodation, award-winning service and a prime location in the city's Heliopolis district. Whether you're in town to attend an event at the Cairo International Convention Centre or for a holiday with your family, you'll find everything you need here to relax and feel invigorated. Our hotel's 440 immaculately appointed rooms and suites envelop you in comfort, with refined amenities including pillowtop bedding, luxurious marble bathrooms and complimentary Wi-Fi. Treat yourse

{'brand': 'JW Marriott', 'name': 'JW Marriott Hotel Chengdu', 'url': 'https://www.marriott.com/CTUMJ', 'street': 'No 19 Dongyu Street, Jinjiang District', 'locality': 'Chengdu', 'state': '', 'postalcode': '610020', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/ctumj-jw-marriott-hotel-chengdu/?directPageRequest=true', 'coordinate3': '30.65681', 'coordinate4': '104.070995', 'description': "[ Iconic Tianfu Square, the symbol of the city, is now home to a symbol of luxury: JW Marriott Hotel Chengdu. We welcome visitors to southwest China's city with refined style, culturally inspired dining and holistic wellbeing. Our casually elegant hotel rooms and suites boast warm, modern decor, spacious bathrooms and mesmerizing views of downtown. Our on-site restaurants celebrate the flavors of China, providing a modern twist on the traditional cuisine found along Chunxi Road, Chengdu's busiest street. We feature eight sophisticated venues, including a 10,635-s

{'brand': 'JW Marriott', 'name': 'JW Marriott Hotel Macau', 'url': 'https://www.marriott.com/MFMJW', 'street': 'Galaxy Macau™, Estrada da Baía da Nossa Senhora da Esperanca s/n, COTAI', 'locality': 'Macau', 'state': '', 'postalcode': '', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/mfmjw-jw-marriott-hotel-macau/?directPageRequest=true', 'coordinate3': '22.1467248', 'coordinate4': '113.5554555', 'description': "[ Luxury knows no bounds at JW Marriott Hotel Macau. We feature tastefully decorated, spacious accommodation, a wealth of 5-star amenities and unbeatable placement within Galaxy Macau™, home to world-class shopping, dining and entertainment. Unwind in 5-star style in your modern room or suite; each one boasts top-quality amenities like luxury bedding, a flat-screen TV, a marble bathroom, a mini-fridge, free Wi-Fi access and floor-to-ceiling windows, which provide sweeping panoramic views of Macau. Visit our award-winning hotel's spectacula

{'brand': 'JW Marriott', 'name': 'JW Marriott Hotel Shanghai at Tomorrow Square', 'url': 'https://www.marriott.com/SHAJW', 'street': '399 Nanjing West Road, Huangpu District', 'locality': 'Shanghai', 'state': '', 'postalcode': '200003', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/shajw-jw-marriott-hotel-shanghai-at-tomorrow-square/?directPageRequest=true', 'coordinate2': '31.230711', 'coordinate3': '121.469886', 'coordinate4': 'N/A', 'description': "[ JW Marriott Hotel Shanghai at Tomorrow Square rises 60 stories above downtown Puxi and is located in the heart of Shanghai’s commercial and shopping district and within easy reach of the city’s many leisure attractions such as Shanghai Museum, Shanghai History Museum, Shanghai Grand Theatre and the Bund. JW Marriott Hotel Shanghai at Tomorrow Square's 333 hotel rooms offer captivating views of the city, enveloping guests in the lavish appointments and business-savvy amenities. While here, guests can embark on a culinary jo

{'brand': 'JW Marriott', 'name': 'JW Marriott Hotel Bengaluru', 'url': 'https://www.marriott.com/BLRJW', 'street': '24/1 Vittal Mallya Road', 'locality': 'Bengaluru', 'state': '', 'postalcode': '560001', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/blrjw-jw-marriott-hotel-bengaluru/?directPageRequest=true', 'coordinate3': '12.972581', 'coordinate4': '77.595199', 'description': "[ Discover quiet luxury and exceptional city centre convenience in the heart of Bangalore at JW Marriott Hotel Bengaluru. Situated just steps from the green gardens of Cubbon Park and only moments from the UB City business district, Koramangala and Mahatma Ghandi (MG) Road, our 5-star hotel provides an enviable location from which to discover Bangalore's top attractions. After exploring the area, unpack in spacious accommodations with stylish rooms, pillowtop bedding, marble bathrooms and private balconies before exploring the wealth of amenities at our hotel. Enjoy acces

...

{'brand': 'JW Marriott', 'name': 'JW Marriott Chicago', 'url': 'https://www.marriott.com/CHIJW', 'street': '151 West Adams Street', 'locality': 'Chicago', 'state': 'Illinois', 'postalcode': '60603', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/chijw-jw-marriott-chicago/?directPageRequest=true', 'coordinate3': '41.879313', 'coordinate4': '-87.632999', 'description': "[ Embark on an extravagant escape from the everyday at JW Marriott Chicago, a landmark hotel brimming with history. Benefit from our hotel's excellent location in downtown Chicago, IL, on the Loop. We offer guests easy access to the West Loop, Columbia College, the Art Institute of Chicago, Millennium Park and Skydeck Chicago. After a busy day in the city, pamper yourself with our array of luxury amenities, including an on-site spa, an indoor lap pool, a modern fitness center and a delicious restaurant and lounge serving authentic Italian cuisine and specialty cocktails. If you're se

{'brand': 'JW Marriott', 'name': 'JW Marriott Essex House New York', 'url': 'https://www.marriott.com/NYCEX', 'street': '160 Central Park South', 'locality': 'New York', 'state': 'New York', 'postalcode': '10019', 'coordinate1': 'Reviews', 'coordinate2': ' View Rates ', 'coordinate3': '', 'coordinate4': ' View Rates ', 'description': '[ Discover the rich history and exquisite Art Deco-inspired decor of JW Marriott Essex House New York. Our luxury hotel is conveniently located adjacent to Central Park South and minutes from Lincoln Center, Columbus Circle and Fifth Avenue, as well as many other attractions. Distinctive, well-appointed rooms and suites feature Art Deco touches, plush bedding, marble bathrooms, mini-bars, ample desks and Wi-Fi. Spacious family connector rooms and suites benefit from expanded living areas and suites with terraces overlooking Central Park are available, as are pet-friendly accommodations. Replenish with a relaxing spa treatment, work out in our modern fitne

{'brand': 'JW Marriott', 'name': 'JW Marriott San Antonio Hill Country Resort & Spa', 'url': 'https://www.marriott.com/SATJW', 'street': '23808 Resort Parkway', 'locality': 'San Antonio', 'state': 'Texas', 'postalcode': '78261', 'coordinate1': 'Reviews', 'coordinate2': ' View Rates ', 'coordinate3': '', 'coordinate4': ' View Rates ', 'description': "[ Now Open. Nestled amid 600 acres of oak-covered hills, JW Marriott San Antonio Hill Country Resort and Spa is a luxury haven within a stunning landscape. Explore the area's most popular attractions, including wineries, SeaWorld, Shops at La Cantera, Natural Bridge Caverns and Six Flags Fiesta Texas. Or return to our hotel for the thrilling River Bluff Water Experience, a nine-acre water park featuring a 1,100-foot lazy river. Celebrate and succeed in 268,000 square feet of sophisticated event space, featuring outdoor pavilions and elegant ballrooms. After a full day of business or sightseeing in San Antonio, TX, savor delectable fare at o

{'brand': 'JW Marriott', 'name': 'JW Marriott Hotel Lima', 'url': 'https://www.marriott.com/LIMDT', 'street': 'Malecon De La Reserva 615, Miraflores', 'locality': 'Lima', 'state': '', 'postalcode': 'Lima 18', 'coordinate1': ' View Rates ', 'coordinate2': '', 'coordinate3': ' View Rates ', 'coordinate4': 'https://www.marriott.com/hotels/maps/travel/limdt-jw-marriott-hotel-lima/?directPageRequest=true', 'description': '[ A luxury escape awaits you at our waterfront JW Marriott Hotel Lima. Situated within steps of the beach and Costa Verde, our hotel is also convenient to Lima attractions like the Cathedral of Lima and Larcomar entertainment center. Our upscale hotel offers elegant rooms with plush beds and crisp linens. Spacious rooms and suites provide ocean views. With over 15,000 square feet of event space, our hotel can host everything from large wedding to corporate conferences. Enjoy delicious Asian and Peruvian cuisine at our restaurants and stop by the lounge for a drink. Our Hea

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,JW Marriott,JW Marriott Hotel Cairo,https://www.marriott.com/CAIJW,"Ring Road- Mirage City- P.O.Box 427, Heliopolis",Cairo,,11757,Reviews,https://www.marriott.com/hotels/maps/travel/ca...,30.071958,31.43415,[ The luxury JW Marriott Hotel Cairo offers a ...,N/A,4.6,4.5
1,JW Marriott,JW Marriott Absheron Baku,https://www.marriott.com/GYDJW,674 Azadliq Square,Baku,,AZ-1010,,View Rates,https://www.marriott.com/hotels/maps/travel/gy...,40.374359,[ Discover 5-star service at JW Marriott Abshe...,N/A,N/A,N/A
2,JW Marriott,JW Marriott Hotel Zhejiang Anji,https://www.marriott.com/HGHAJ,No 1 Raocheng South Line,Anji,,313300,Reviews,https://www.marriott.com/hotels/maps/travel/hg...,30.614168,119.707498,[ Discover towering bamboo forests and green m...,N/A,4.7,4.5
3,JW Marriott,JW Marriott Hotel Beijing,https://www.marriott.com/BJSJW,"83 Jian Guo Road, Chaoyang District",Beijing,,100025,Reviews,https://www.marriott.com/hotels/maps/travel/bj...,39.909433,116.476003,[ Discover the luxurious accommodations at JW ...,N/A,4.7,4.5
4,JW Marriott,JW Marriott Hotel Beijing Central,https://www.marriott.com/BJSJC,"18 Xuanwumenwai Street, Xicheng District",Beijing,,100052,Reviews,https://www.marriott.com/hotels/maps/travel/bj...,39.89532,116.36827,[ JW Marriott Hotel Beijing Central enjoys a p...,N/A,4.6,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,JW Marriott,JW Marriott Hotel Rio de Janeiro,https://www.marriott.com/RIOMC,"Avenida Atlantica, 2600, Copacabana",Rio de Janeiro,,22041-001,Reviews,https://www.marriott.com/hotels/maps/travel/ri...,-22.972838,-43.185807,[ Experience a blend of luxury and comfort at ...,N/A,4.5,4.5
99,JW Marriott,JW Marriott Hotel Bogota,https://www.marriott.com/BOGJW,Calle 73 Nr 8-60,Bogota,,,4.6569,-74.055625,N/A,N/A,[ Take your travel experience to the next leve...,N/A,4.8,4.8
100,JW Marriott,JW Marriott Hotel Quito,https://www.marriott.com/UIODT,Avenida Orellana 1172 y Avenida Amazonas,Quito,,,Reviews,https://www.marriott.com/hotels/maps/travel/ui...,-0.197045,-78.489109,[ Elevate your Ecuador adventures at JW Marrio...,N/A,4.7,4.6
101,JW Marriott,JW Marriott El Convento Cusco,https://www.marriott.com/CUZMC,Esquina de la Calle Ruinas 432 y San Agustin,Cusco,,,View Rates,,View Rates,https://www.marriott.com/hotels/maps/travel/cu...,[ Embrace a truly unique stay at JW Marriott E...,N/A,N/A,N/A


In [6]:
hotel_df = hotel_df.drop_duplicates(subset='name')

In [7]:
hotel_df

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,JW Marriott,JW Marriott Hotel Cairo,https://www.marriott.com/CAIJW,"Ring Road- Mirage City- P.O.Box 427, Heliopolis",Cairo,,11757,Reviews,https://www.marriott.com/hotels/maps/travel/ca...,30.071958,31.43415,[ The luxury JW Marriott Hotel Cairo offers a ...,N/A,4.6,4.5
1,JW Marriott,JW Marriott Absheron Baku,https://www.marriott.com/GYDJW,674 Azadliq Square,Baku,,AZ-1010,,View Rates,https://www.marriott.com/hotels/maps/travel/gy...,40.374359,[ Discover 5-star service at JW Marriott Abshe...,N/A,N/A,N/A
2,JW Marriott,JW Marriott Hotel Zhejiang Anji,https://www.marriott.com/HGHAJ,No 1 Raocheng South Line,Anji,,313300,Reviews,https://www.marriott.com/hotels/maps/travel/hg...,30.614168,119.707498,[ Discover towering bamboo forests and green m...,N/A,4.7,4.5
3,JW Marriott,JW Marriott Hotel Beijing,https://www.marriott.com/BJSJW,"83 Jian Guo Road, Chaoyang District",Beijing,,100025,Reviews,https://www.marriott.com/hotels/maps/travel/bj...,39.909433,116.476003,[ Discover the luxurious accommodations at JW ...,N/A,4.7,4.5
4,JW Marriott,JW Marriott Hotel Beijing Central,https://www.marriott.com/BJSJC,"18 Xuanwumenwai Street, Xicheng District",Beijing,,100052,Reviews,https://www.marriott.com/hotels/maps/travel/bj...,39.89532,116.36827,[ JW Marriott Hotel Beijing Central enjoys a p...,N/A,4.6,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,JW Marriott,JW Marriott Hotel Rio de Janeiro,https://www.marriott.com/RIOMC,"Avenida Atlantica, 2600, Copacabana",Rio de Janeiro,,22041-001,Reviews,https://www.marriott.com/hotels/maps/travel/ri...,-22.972838,-43.185807,[ Experience a blend of luxury and comfort at ...,N/A,4.5,4.5
99,JW Marriott,JW Marriott Hotel Bogota,https://www.marriott.com/BOGJW,Calle 73 Nr 8-60,Bogota,,,4.6569,-74.055625,N/A,N/A,[ Take your travel experience to the next leve...,N/A,4.8,4.8
100,JW Marriott,JW Marriott Hotel Quito,https://www.marriott.com/UIODT,Avenida Orellana 1172 y Avenida Amazonas,Quito,,,Reviews,https://www.marriott.com/hotels/maps/travel/ui...,-0.197045,-78.489109,[ Elevate your Ecuador adventures at JW Marrio...,N/A,4.7,4.6
101,JW Marriott,JW Marriott El Convento Cusco,https://www.marriott.com/CUZMC,Esquina de la Calle Ruinas 432 y San Agustin,Cusco,,,View Rates,,View Rates,https://www.marriott.com/hotels/maps/travel/cu...,[ Embrace a truly unique stay at JW Marriott E...,N/A,N/A,N/A


In [8]:
hotel_df.to_csv('jwmarriott_description.csv')